In [ ]:
from fastai import *          # Quick accesss to most common functionality
from fastai.tabular import *  # Quick accesss to tabular functionality     # Access to example data provided with fastai

# Tabular example

Tabular data should be in a Pandas `DataFrame`.

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
dep_var = '>=50k'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [ ]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [ ]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test, label=0)
                           .databunch())

In [ ]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
Private,Some-college,Never-married,Other-service,Own-child,White,False,-1.5090,-0.2272,-0.0312
Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,0.9831,-0.9498,-0.4224
?,5th-6th,Married-civ-spouse,?,Husband,White,False,1.3496,1.0465,-2.7692
Private,HS-grad,Divorced,Exec-managerial,Not-in-family,White,False,0.4701,-1.1081,-0.4224
Self-emp-not-inc,Prof-school,Never-married,Prof-specialty,Not-in-family,White,False,0.6166,0.0885,1.9245
Private,HS-grad,Married-civ-spouse,Other-service,Husband,White,False,-0.6294,-0.7424,-0.4224
Private,Some-college,Never-married,Handlers-cleaners,Not-in-family,White,False,-1.3624,-1.5711,-0.0312
Private,HS-grad,Divorced,Craft-repair,Unmarried,White,False,-0.1896,0.1327,-0.4224
Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,False,-0.7760,0.7006,1.1422
Local-gov,HS-grad,Married-civ-spouse,Protective-serv,Husband,White,False,-0.5561,0.5114,-0.4224


In [ ]:
learn = get_tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit(1, 1e-2)

Total time: 00:05
epoch  train_loss  valid_loss  accuracy
1      0.358344    0.383886    0.820000  (00:05)



In [ ]:
def test_accuracy(learn):
    assert learn.validate()[1] > 0.7

def test_same_categories(learn):
    x_train,y_train = learn.data.train_ds[0]
    x_valid,y_valid = learn.data.valid_ds[0]
    x_test,y_test = learn.data.test_ds[0]
    assert x_train.classes.keys() == x_valid.classes.keys()
    assert x_train.classes.keys() == x_test.classes.keys()
    for key in x_train.classes.keys():
        assert np.all(x_train.classes[key] == x_valid.classes[key])
        assert np.all(x_train.classes[key] == x_test.classes[key])
        
def test_same_fill_nan(learn):
    df = pd.read_csv(path/'adult.csv')
    nan_idx = np.where(df['education-num'].isnull())
    val = None
    for i in nan_idx[0]:
        x,y = (learn.data.train_ds[i] if i < 800 else learn.data.valid_ds[i-800])
        j = x.names.index('education-num') - len(x.cats)
        if val is None: val = x.conts[j]
        else: assert val == x.conts[j]
        if i >= 800:
            x,y = learn.data.test_ds[i-800]
            assert val == x.conts[j]
            
def test_normalize(learn):
    df = pd.read_csv(path/'adult.csv')
    train_df = df.iloc[0:800].append(df.iloc[1000:])
    c = 'age'
    mean, std = train_df[c].mean(), train_df[c].std()
    for i in np.random.randint(0,799, (20,)):
        x,y = learn.data.train_ds[i]
        assert np.abs(x.conts[0] - (df.loc[i, c] - mean) / (1e-7 + std)) < 1e-6
    for i in np.random.randint(800,1000, (20,)):
        x,y = learn.data.valid_ds[i-800]
        assert np.abs(x.conts[0] - (df.loc[i, c] - mean) / (1e-7 + std)) < 1e-6
    for i in np.random.randint(800,1000, (20,)):
        x,y = learn.data.test_ds[i-800]
        assert np.abs(x.conts[0] - (df.loc[i, c] - mean) / (1e-7 + std)) < 1e-6

In [ ]:
test_accuracy(learn)

In [ ]:
test_same_categories(learn)

In [ ]:
test_same_fill_nan(learn)

In [ ]:
test_normalize(learn)